# Load Models AND Orm

In [ ]:
from model.models import Base, session, engine, Parti, Candidat, CandidatParti, ResultatCondidatParti, ResultatMetaInfo, UrneVote

## Create Schema

In [ ]:
Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)

# Load CSVs

In [ ]:
from glob import glob
import pandas as ps

In [ ]:
files = glob("data/legis/*.csv")

## Processing

## Helpers

In [ ]:
parti_potentiels = {
    "EDroit": set(["RN","FN","DLF","DLR","FRS","PCD","MC","SC","CNIP","UPR","LP","CJ","LS","SIEL","SIEL","PI","LI","BI","DI","PDF","MNR","AF","PNF","T&P ou TP","RF","E&R","AR"]),
    "DROIT": set(["LR","UMP","SL","LR","LMR","CPNT","LT-NE","ASFE"]),
    "CENTRE": set(["LREM","EM","AC","PE","UCE","MoDem","LR","TdP ou TDP","EC","PRV ou RAD","LGM","LFA","UDI","FED","CER","LC","NC","MEI","LV","MHAN","AEI","LEF","NC","GC","TEM","PFE","EDE"]),
    "GAUCHE": set(["PS","PE","EELV","G·s","GE","MDP ou MdP","LND","PP","ND","PRG","LRDG","NGS","GRS","Après","MRC","MDC","NAR","LFD13","PA","UDMF","PP","PP","alias Cindy Lee","RS","SP","IDG"]),
    "EGAUCHE": set(["LFI","PG","ENS","REV","PCF","NPA","LO","PPLD","POI","PT","POID","LTF","PCOF","PRCF","UCL","FA","IR","PIR","RP"])
}

In [ ]:
parti_courants = ps.read_csv("data/ptendance.csv")
for index, row in parti_courants.iterrows():
    parti_potentiels[row['tendance']].add(row['parti'])

In [ ]:
def extract_year_file(file_name):
    return file_name.split('cdsp_presi')[1].split('t')[0]

In [ ]:
def extract_tendance_from_parti(parti):
    for key, value in parti_potentiels.items():
        if parti.upper() in value:
            return key

    return None

In [ ]:
def extract_candidat_part(str):
    sp = str.split(" (")
    if(len(sp) > 1):
        return (sp[0], sp[1].replace(")", ""), str)
    else:
        return (sp[0], "NEUTRE", str)

In [ ]:
def save_regions(df):
    regio_df = df.filter(regex='département')
    regio_df = (
        regio_df
        .rename(columns=lambda x: 'department_code' if 'code' in x.lower() else 'department_name')
        )
    regio_df = regio_df.drop_duplicates()
    regio_df.to_sql('region',con=engine, if_exists='replace', index=False)

In [ ]:
def handle_term(df, is_final, year):
    candidat_partis = load_candidat_parti(df)
    save_regions(df)
    df.rename(columns=lambda x: 'region_id' if ('code' in x.lower() and 'département' in x.lower()) else x, inplace = True)
    df.rename(columns=lambda x: 'circonscription' if 'circonscription' in x.lower() else x, inplace = True)

    for _, row in df.iterrows():
        urne_Vote = UrneVote(
            circonscription=row['circonscription'], 
            region_id=row['region_id'],
            annee=year,
            final_round=is_final,
            is_legis=False
        )
        session.add(urne_Vote)
        session.flush()
        for header, candidat_obj in candidat_partis.items():
            candidat_result = ResultatCondidatParti(
                    urne_vote_id = urne_Vote.id,
                    candidat_parti = candidat_obj.id,
                    value = row[header]
                )
            session.add(candidat_result)
            result_meta_info = ResultatMetaInfo(
                urne_vote_id=urne_Vote.id,
                inscripts=row['Inscrits'],
                votants=row['Votants'],
                nullparts=row['Blancs et nuls'],
                exprimes=row['Exprimés']
            )
            session.add(result_meta_info)
        
    session.commit()

In [ ]:
def load_candidat_parti(df):
    candidats = [extract_candidat_part(x) for x in df.columns if x.isupper()]
    result = {}
    for (candidat_name, parti_name, header) in candidats:
        tendance = extract_tendance_from_parti(parti_name)
        candidat = session.query(Candidat).filter_by(candidat_name = candidat_name)
        if candidat.count() == 0:
            candidat = Candidat(candidat_name = candidat_name)
        else:
            candidat = candidat.one()

        session.add(candidat)
        parti = session.query(Parti).filter_by(parti_name=parti_name)
        if parti.count() == 0:
            parti = Parti(parti_name=parti_name)
        else:
            parti = parti.one()

        session.add(parti)
        candidat_partie = session.query(CandidatParti).filter_by(candidat=candidat, parti=parti, courant=tendance)
        if candidat_partie.count() == 0:
            candidat_partie = CandidatParti(candidat=candidat, parti=parti, courant=tendance)
        else:
            candidat_partie = candidat_partie.one()

        session.add(candidat_partie)
        result[header] = candidat_partie

    session.commit()
    return result

## First Round

In [ ]:
def firstTerm(df, year):
    handle_term(df, year=year,is_final=False)

## Second Round

In [ ]:
def secondTerm(df, year):
    handle_term(df, year=year,is_final=True)

## Main Proccess

In [ ]:
for file in files:
    result = ps.read_csv(file)
    _, tail = os.path.split(file)
    year = extract_year_file(tail)
    if tail.find("t1_circ") == -1:
        secondTerm(result,year=year)
    else:
        firstTerm(result,year=year)